<a href="https://colab.research.google.com/github/alex-bolshunov/Data_mining_project/blob/main/data_mining_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [396]:
#columns: npi, gender, state, dredentials, speciality, opiods, non opiods, risk skale (overdose / population) ?
#https://www.kaggle.com/datasets/apryor6/us-opiate-prescriptions/data?select=overdoses.csv

In [397]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [398]:
#get data
!curl "https://raw.githubusercontent.com/alex-bolshunov/Data_mining_project/main/prescriber_info.csv" -o prescriber.csv
!curl "https://raw.githubusercontent.com/alex-bolshunov/Data_mining_project/main/overdoses.csv" -o overdose.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13.7M  100 13.7M    0     0  60.7M      0 --:--:-- --:--:-- --:--:-- 61.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1789  100  1789    0     0  16470      0 --:--:-- --:--:-- --:--:-- 16564


In [399]:
#overdose df
overdose_df = pd.read_csv("overdose.csv")

In [400]:
overdose_column_names = ["state_full", "pop", "od", "state"]
overdose_df_types = {
    overdose_column_names[1]: "int32",
    overdose_column_names[2]: "int32",
}

In [401]:
#renaming columns
overdose_df.columns = overdose_column_names

In [402]:
#removing comas
num_columns_to_clean = ["pop", "od"]
overdose_df[num_columns_to_clean] = overdose_df[num_columns_to_clean].map(lambda s: s.replace(",", ""))

In [403]:
#changing datatype
overdose_df = overdose_df.astype(overdose_df_types)

In [404]:
#subset three columns, remove full state name column
overdose_df = overdose_df.loc[:, overdose_column_names[1:]]

In [405]:
#create new variable overdose ratio (overdose / populaiton)
overdose_df["od_ratio"] =  (overdose_df["od"]/overdose_df["pop"]*100)

In [406]:
#summary statistics
overdose_df["od_ratio"].agg(["min", "mean", "median", "max"])

,od_ratio
min,0.005944
mean,0.015944
median,0.014906
max,0.033813


In [407]:
#define the group based on the number
def categorize(num):
  available_groups = "ABC"
  selected_group = ""

  if(num < 0.01): selected_group = available_groups[2]
  elif(num < 0.02): selected_group = available_groups[1]
  else: selected_group = available_groups[0]

  return selected_group

In [408]:
#create variabe od group
overdose_df["od_group"] = overdose_df["od_ratio"].apply(categorize)

In [409]:
#check value counts
overdose_df["od_group"].value_counts().sort_index()

,count
od_group,
A,10
B,34
C,6


In [410]:
overdose_df.iloc[25:30]

,pop,od,state,od_ratio,od_group
25,1015165,125,MT,0.012313,B
26,1868516,125,NE,0.006690,C
27,2790136,545,NV,0.019533,B
28,1323459,334,NH,0.025237,A
29,8899339,1253,NJ,0.014080,B


In [411]:
#testing values
groups = "ABC"
groups_states_dict = {}

#populate dict state:group pairs
for char in groups:
  groups_states_dict.update({
     key:char for key in overdose_df[overdose_df["od_group"] == char]["state"].to_list()
  })

#check if the state in correct group
def check_group(state, group):
  if(group != groups_states_dict.get(state, "")): raise Exception("Group doesn't match")

#loop over values check each pair
def test1(df):
  for state, group in list(df[['state', "od_group"]].itertuples(index=False, name=None)): check_group(state, group)
  print("Test completed")


In [412]:
#prescriber df
prescriber_df = pd.read_csv("prescriber.csv")

In [413]:
prescriber_df.head()

,NPI,Gender,State,Credentials,Specialty,ABILIFY,ACETAMINOPHEN.CODEINE,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,...,VERAPAMIL.ER,VESICARE,VOLTAREN,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,Opioid.Prescriber
0,1710982582,M,TX,DDS,Dentist,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1245278100,F,AL,MD,General Surgery,0,0,0,0,0,...,0,0,0,0,0,0,0,0,35,1
2,1427182161,F,NY,M.D.,General Practice,0,0,0,0,0,...,0,0,0,0,0,0,0,0,25,0
3,1669567541,M,AZ,MD,Internal Medicine,0,43,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1679650949,M,NV,M.D.,Hematology/Oncology,0,0,0,0,0,...,0,0,0,0,17,28,0,0,0,1


In [414]:
#create new variable sum
start_col = "ABILIFY"
end_col = "ZOLPIDEM.TARTRATE"

prescriber_df["sum"] = prescriber_df.loc[:, start_col: end_col].apply("sum", axis=1)

In [415]:
#subset values, change names of the columns
prescriber_df_select_cols = ["NPI", "Gender", "State", "Credentials", "Specialty", "Opioid.Prescriber", "sum"]
prescribeer_df_new_cols = pd.Series(prescriber_df_select_cols).apply(lambda s: s.lower()).to_list()
prescriber_df = prescriber_df[prescriber_df_select_cols].copy()
prescriber_df.columns = prescribeer_df_new_cols

In [416]:
prescriber_df.head()

,npi,gender,state,credentials,specialty,opioid.prescriber,sum
0,1710982582,M,TX,DDS,Dentist,1,29
1,1245278100,F,AL,MD,General Surgery,1,1418
2,1427182161,F,NY,M.D.,General Practice,0,289
3,1669567541,M,AZ,MD,Internal Medicine,1,1757
4,1679650949,M,NV,M.D.,Hematology/Oncology,1,179


In [417]:
#merge 2 dataframes
df = pd.merge(prescriber_df, overdose_df, on = "state")

#check value counts for overdose groups
df["od_group"].value_counts().sort_index()

,count
od_group,
A,3694
B,18527
C,2459


In [418]:
#generate 5 random indexes
num_index = 5
df_len = len(df)

indexes_check  = (np.random.default_rng().random((num_index,)) * df_len).round().astype('int').tolist()

In [419]:
df.iloc[indexes_check]

,npi,gender,state,credentials,specialty,opioid.prescriber,sum,pop,od,od_ratio,od_group
21469,1588086672,M,SC,NaN,Physician Assistant,1,14,4774839,701,0.014681,B
9108,1437594306,F,FL,M.D,General Practice,1,1694,19552860,2634,0.013471,B
6110,1811095342,F,WI,MD,Dermatology,1,416,5742713,853,0.014854,B
20007,1568781706,M,NC,M.D.,Student in an Organized Health Care Education/...,0,51,9848060,1358,0.013790,B
2943,1952463119,F,NY,MD,Psychiatry,0,647,19651127,2300,0.011704,B


In [420]:
#test state group pairs if correct
test1(df)

Test completed


In [421]:
#subset columns
#similar dataset will be used to train the model
#sum column is generic and can be subsitution with different variations (opiods/non-opiods, etc)

df_columns = ["credentials", "specialty", "opioid.prescriber", "sum", "od_group"]

df = df[df_columns].copy()

df.iloc[indexes_check]

,credentials,specialty,opioid.prescriber,sum,od_group
21469,NaN,Physician Assistant,1,14,B
9108,M.D,General Practice,1,1694,B
6110,MD,Dermatology,1,416,B
20007,M.D.,Student in an Organized Health Care Education/...,0,51,B
2943,MD,Psychiatry,0,647,B
